# 統計的検定の選び方ガイド（デシジョンツリー）

「どの検定を使えばいいのかわからない」という場面で役立つ、インタラクティブな検定選択ツールです。
データの種類や比較対象（群数、対応の有無）を選択していくと、適切な検定手法が提案されます。

## 観点の棚卸し（検定選択のポイント）

検定手法を選ぶ際に確認すべき主要なポイントは以下の3点です。

| 観点 | 選択肢の例 | 影響する手法 |
|---|---|---|
| **1. データの尺度** | 連続量 vs 離散量(カテゴリ) | t検定 vs カイ二乗検定 |
| **2. 群の数と対応** | 1群 vs 2群(対応あり/なし) vs 3群以上 | t検定 vs ANOVA vs Paired-t |
| **3. 分布の仮定** | 正規分布に従う(パラメトリック) vs 従わない(ノンパラ) | t検定 vs マン・ホイットニーのU検定 |

**補足**
- **パラメトリック検定**: データが特定の分布（主に正規分布）に従うことを仮定します。検出力が高いですが、前提が崩れると信頼できません。
- **ノンパラメトリック検定**: 分布の形を仮定しません（順位などを使う）。汎用性が高いですが、検出力はやや劣ります。

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# 日本語フォントの設定（必要に応じて）
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = '../fonts/ipaexg.ttf'
try:
    fm.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'IPAexGothic'
except:
    pass

# 質問と選択肢の定義
questions = {
    'step1': {
        'question': '1. データの種類（目的変数）は？',
        'options': [
            ('連続量（身長、売上、テストの点数など）', 'continuous'),
            ('離散量/カテゴリ（○×、A/B/C、人数、回数）', 'discrete')
        ]
    },
    'continuous_step2': {
        'question': '2. 比較するグループ（群）の数は？',
        'options': [
            ('1つのグループ（理論値との比較）', '1_group'),
            ('2つのグループ（A組とB組の比較など）', '2_groups'),
            ('3つ以上のグループ', '3_groups')
        ]
    },
    '2_groups_step3': {
        'question': '3. データに対応（ペア）はあるか？',
        'description': '「対応あり」とは、同じ人から「投薬前・投薬後」のように2回データを取った場合など。',
        'options': [
            ('対応なし（独立した2群：Aさん達とBさん達）', 'unpaired'),
            ('対応あり（ペアデータ：Aさんの事前と事後）', 'paired')
        ]
    },
    'discrete_step2': {
        'question': '2. どのような比較をしたい？',
        'options': [
            ('比率の差（クリック率など）', 'proportion'),
            ('分布の適合度・独立性（サイコロの目、クロス集計）', 'chi2')
        ]
    }
}

# 診断ロジック
def get_recommendation(state):
    data_type = state.get('step1')
    
    if data_type == 'continuous':
        n_groups = state.get('continuous_step2')
        
        if n_groups == '1_group':
            return """### 推奨される検定：1標本t検定 (One-sample t-test)
            - 母分散が既知なら **Z検定** も可。
            - 正規性が疑わしい場合は **ウィルコクソンの符号順位検定**"""
            
        elif n_groups == '2_groups':
            pairing = state.get('2_groups_step3')
            if pairing == 'unpaired':
                return """### 推奨される検定：t検定 (Unpaired t-test / Welch's t-test)
                - 等分散性が仮定できるならスチューデントのt検定。
                - 等分散性が怪しいなら **ウェルチのt検定**（こちらが安全で推奨されることが多い）。
                - 正規性がない場合は **マン・ホイットニーのU検定**。"""
            elif pairing == 'paired':
                return """### 推奨される検定：対応のあるt検定 (Paired t-test)
                - 正規性がない場合は **ウィルコクソンの符号順位検定**。"""
                
        elif n_groups == '3_groups':
            return """### 推奨される検定：分散分析 (ANOVA)
            - 正規性がない場合は **クラスカル・ウォリス検定**。
            - 差があることがわかったら、多重比較（テューキー法など）を行う。"""
            
    elif data_type == 'discrete':
        disc_type = state.get('discrete_step2')
        
        if disc_type == 'proportion':
            return """### 推奨される検定：比率の差の検定 (Z-test for proportions)
            - サンプル数が少ない場合は **フィッシャーの正確確率検定**。"""
        elif disc_type == 'chi2':
            return """### 推奨される検定：カイ二乗検定 (Chi-square test)
            - 適合度検定 または 独立性の検定として利用。"""
            
    return "(選択してください)"

class TestSelector:
    def __init__(self):
        self.state = {}
        self.output_area = widgets.Output()
        self.widgets = {}
        
        # ステップ1の表示
        self.show_step('step1')
        
    def show_step(self, step_key):
        q_data = questions[step_key]
        
        label = widgets.Label(value=q_data['question'])
        options = q_data['options']
        
        dropdown = widgets.Dropdown(
            options=[('選択してください', None)] + options,
            value=None,
            description='回答:',
        )
        
        def on_change(change):
            if change['type'] == 'change' and change['name'] == 'value':
                val = change['new']
                if val is None: return
                
                self.state[step_key] = val
                self.clear_following_steps(step_key)
                self.next_step(step_key, val)

        dropdown.observe(on_change)
        
        container = widgets.VBox([label, dropdown])
        if 'description' in q_data:
            container.children = (label, widgets.HTML(f"<small>{q_data['description']}</small>"), dropdown)
            
        self.widgets[step_key] = container
        self.refresh_display()

    def clear_following_steps(self, current_step):
        # 簡易的に、現在のステップ以降に登録されたウィジェットを削除して再描画
        # （辞書のキー順序に依存するが、今回は単純なツリーなので簡易実装）
        keys = list(self.widgets.keys())
        if current_step in keys:
            idx = keys.index(current_step)
            for k in keys[idx+1:]:
                del self.widgets[k]
                if k in self.state:
                    del self.state[k]
                    
    def next_step(self, current_step, val):
        next_key = None
        
        if current_step == 'step1':
            if val == 'continuous': next_key = 'continuous_step2'
            elif val == 'discrete': next_key = 'discrete_step2'
            
        elif current_step == 'continuous_step2':
            if val == '2_groups': next_key = '2_groups_step3'
            # 1_group, 3_groups は終了
            
        if next_key:
            self.show_step(next_key)
        else:
            # 診断完了
            self.show_result()

    def show_result(self):
        result_text = get_recommendation(self.state)
        self.widgets['result'] = widgets.Output()
        with self.widgets['result']:
            display(Markdown(result_text))
        self.refresh_display()

    def refresh_display(self):
        self.output_area.clear_output()
        with self.output_area:
            for k, w in self.widgets.items():
                display(w)

selector = TestSelector()
display(selector.output_area)